# Ejemplo. Herencia.

## 1. Objetivo(s)

* Entender la implemenación herencia simple.
* Utilizar herencia para extender las funcionalidad de clases built-in de Python.
* Utilizar la sobreescritura de métodos y `__super()__`.

## 2. Desarrollo:

A continuación se muestra un ejemplo donde se implementa la herencia. Se proponr realiazar un administrador de contactos que registre nombres y correo electrónico de muchas personas. La clase `Contact` es responsable de mantener una lista de todos los contactos.

In [1]:
from typing  import List

class Contact():

    all_contacts: List["Contact"] = []

    def __init__(self, name : str, email : str) -> None:
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name}, {self.email})"

En este ejemplo se introduce el concepto de __variables de clase__. La lista `all_contacts` es parte de la definición de la clases, pero es compartida por todas las instancias de esta clase. Si un campo no puede ser encontrada a través de `self`, entonces será encontrado en la clases y por lo tanto se referira a una sola lista.

Advertencia: Si se intenta acceder a `all_contacts` mediante `self.all_contacts`, en realidad se estará creando una nueva instancia de una variable asociada con el objeto. Las variables de clase estará sin cambios y accesible a través de `Contacs.all_contacts`

In [2]:
c_1 = Contact("Dusty", "dusty@example.com")
c_2 = Contact("Brayan", "elbrayan@itmaybeahack.com")

Contact.all_contacts


[Contact(Dusty, dusty@example.com), Contact(Brayan, elbrayan@itmaybeahack.com)]

Para ejemploficar la herencia, supongamos que se tiene un tipo de contacto `Supplier` al cual se le puede solicitar una orden. Si se define el método `order()` en la clase `Contact`se podría cometer el error de realizar una orden a otro tipo de contacto como: un amigo, cliente, familia, etc. Por lo que, se creara una clase `Supplier` que actua como la clase `Contact` pero que en adición cuenta con un método `order`.


In [3]:
class Supplier(Contact):
    def order(self, order : "Order") -> None:
        print(f"{order} order to {self.name}")

In [6]:
c_1 = Contact("Dusty", "dust@email.com")
s_1 = Supplier("Steve", "steve@email.com")

print(c_1, s_1)

Contact(Dusty, dust@email.com) Supplier(Steve, steve@email.com)


In [7]:
print(c_1.all_contacts)

[Contact(Dusty, dusty@example.com), Contact(Brayan, elbrayan@itmaybeahack.com), Contact(Dusty, dust@email.com), Supplier(Steve, steve@email.com), Contact(Dusty, dust@email.com), Supplier(Steve, steve@email.com), Contact(Dusty, dust@email.com), Supplier(Steve, steve@email.com)]


In [8]:
from pprint import pprint

pprint(c_1.all_contacts)

[Contact(Dusty, dusty@example.com),
 Contact(Brayan, elbrayan@itmaybeahack.com),
 Contact(Dusty, dust@email.com),
 Supplier(Steve, steve@email.com),
 Contact(Dusty, dust@email.com),
 Supplier(Steve, steve@email.com),
 Contact(Dusty, dust@email.com),
 Supplier(Steve, steve@email.com)]


In [9]:
s_1.order("I need")

I need order to Steve


In [10]:
c_1.order("I need")

AttributeError: 'Contact' object has no attribute 'order'

## Extensión de clases built-ins

Uno de los aspectos prácticos de la herencia es que permite agregar funcioalidad a clases built-in de Python. Para ejemplificar, pensemos en la clase anterior `Contact` la cual permite agregar los contactos a una lista ¿Qué pasaría si se quisiera realizar una búsqueda de un nombre? Se podría agregar un método a la clase contacto para realizar la busqueda, pero en realidad este método pertenecería a la lista como tal y no a la clase `Contact`.

Para resolver esto, se utilizará la herencia sobre un tipo build-in, es decir, sobre el tipo `list`.



In [11]:
from __future__ import annotations

class ContactList(list["Contact"]):
    def search(self, name: str) -> list["Contact"]:

        matching_contacts : list["Contact"] = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts

In [12]:
class Contact():

    all_contacts = ContactList()

    def __init__(self, name : str, email : str) -> None:
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name}, {self.email})"

In [13]:
c1 = Contact("John A", "john@example.net")
c2 = Contact("John B", "john@sloop.net")
c1 = Contact("Jenna C", "cutty@sark.io")

[c.name for c in Contact.all_contacts.search('John')]

['John A', 'John B']

# Sobreescritura y `__super_()`

La herencia permite crear una nueva clase (hija) tomando como base otra clase (padre), y agregar nuevas funcionalidades a la clase hija. Sin embargo, en algunas ocasiones será necesario realizar cambio de comportamientos. Por ejemplo, en la clase `Contact` solo permite el nombre y dirección de correo. Pero si quisieramos agregar un número de telefono para los amigos ¿Cómo resolver este problema?

A continuación se muestra la definición de una clase `Friend` la cual sobreescribe el comportamiento del método `__init__()`. Sobreescrbir significa alterar o reemplazar el comportamiento de la superclase o clase padre con un nuevo método (con el mismo nombre) en la subclase o clase hija.

In [14]:
class Friend(Contact):
    def __init__(self, name: str, email: str, phone: str) -> None:
        self.name = name
        self.email = email
        self.phone = phone

Recuerda que cualquier método puede ser sobreescrito, no solo `__init__()`. Sin embargo, antes de continuar, consideremos que existe un problema en este ejemplo anterior. Las clases `Contact` y `Friend` tiene código duplicado al momento de asignar valores a las propiedades `name` y `email`. Recordemos que el código duplicado causa problemas en el mantenimiento ya que se tiene que modificar código en dos o más lugares. En adición, la clase `Friend` no esta agregando los objetos a las lista `all_contacts` como lo hace `Contact`. Por último, se considera el diseño hacia atrás de `Contact`, queremos que las nuevas características se agregen a la clase `Friend`.

La solución es ejecutar el método `__init__()` original de la clase `Contact` dentro de la nueva clase. Esto se puede realizar mediante la función `super()`, la cual retorna el objeto como si fuera instanciado a partir de la clase padre, permitiendo realizar la llamdad de cual quier método padre de forma directa.

In [15]:
class Friend(Contact):
    def __init__(self, name: str, email: str, phone: str) -> None:
        super().__init__(name, email)
        self.phone = phone
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.name}, {self.email}, {self.phone})"

En este ejemplo anterior, aprovechamos para sobre escribir el método `__repr__()` para que se imprima una representación en cadena de los objetos de tipo `Friend` con la información del teléfono.

In [16]:
f = Friend("Dusty", "dusty@private.com", "953123-12345")
Contact.all_contacts

[Contact(John A, john@example.net),
 Contact(John B, john@sloop.net),
 Contact(Jenna C, cutty@sark.io),
 Friend(Dusty, dusty@private.com, 953123-12345)]

Nota: Para estudiantes más experimentados, se recomienda investigar sobre la implementación de herencia multiple. (pa-95).